#### Middleware

###### summarization

In [ ]:
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage, SystemMessage

### message based summarization
agent = create_agent(model="gpt-5-mini", checkpointer=InMemorySaver(), middleware=[
    SummarizationMiddleware(model="gpt-5-nano",trigger=("messages",10), keep=("messages", 4))
])

agent

In [ ]:
questions= [
    "What is 2+2?",
    "What is 10*5?",
    "What is 100/4?",
    "What is 15-7?",
    "What is 3*3?",
    "What is 4*4?",
]

In [ ]:
config = {"configurable":{"thread_id":"1"}}
for q in questions:
    response = agent.invoke({"messages":[HumanMessage(q)]}, config)
    print(f"Messages: {response}")
    print(f"Messages: {len(response['messages'])}")

### Token size

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver


@tool
def search_hotels(city: str)-> str:
    """Search hotels - returns long responses to use more tokens"""
    return f"""Hotels in {city}: 
    1. Grand Hotel - 5 star, $350/night, spa, pool, gym
    2. City Inn - 4 star, $180/night, business center
    3. Budget Stay - 3 star, $75/night, free wifi"""


agent = create_agent(model="gpt-5-mini", checkpointer=InMemorySaver(),tools=[search_hotels], middleware=[
    SummarizationMiddleware(model="gpt-5-nano", trigger=("tokens",550), keep=("tokens", 200))
])
agent

In [ ]:
config = {"configurable":{"thread_id":"1"}}

def count_tokens(messages):
    total_chars = sum(len(str(m.content)) for m in messages)
    return total_chars // 4 # 4 chars = 1 token

# run test
cities = ["Paris", "London","Tokyo", "New York", "Dubia", "Singapore"]

for city in cities:
    response = agent.invoke({"messages": [HumanMessage(content=f"find hotels in {city}")]}, config)
    
    tokens = count_tokens(response['messages'])
    print(f"{city}: ~{tokens} tokens, {len(response['messages'])} messages")
    print(f"{response['messages']}")
